# 2018 Col150 Data Set
## Data and Graphical Analysis

In [1]:
import os, sys, shutil, subprocess
import scipy
from scipy import ndimage
from scipy.spatial import distance

import numpy as np
import pandas as pd

from sklearn.decomposition import PCA

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
matplotlib.rcParams.update({'figure.max_open_warning': 0})
matplotlib.rcParams.update({'font.size': 12})


%matplotlib inline
path = os.getcwd()

In [2]:
Ncirc = 150
ematrix_dict = {0:'tilt', 1:'roll', 2:'twist', 3:'shift', 4:'slide', 5:'rise'}

incons   = ['oring','pcirc']

inseqs1   = ['col'+str(i).zfill(2) for i in range(1, 10, 1)]
inseqs2   = ['A150','G150','A75G75','A15G15','A1G1','A2G2','A3G3','A4G4','A5G5']

seqvars        = ['std','c05a','c15a','c20a','c30a']
seqvars_colors = {'std':'grey','c05a':'blue','c15a':'red','c20a':'gold','c30a':'green'}
seqvars_marks  = {'std':'s','c05a':'<','c15a':'^','c20a':'d','c30a':'P'}

ff_dict       = {'idt':'1-state','org':'2-state','vic':'3-state','vor':'4-state'}

ff_st         = [str(i)+'-state' for i in range(1, 5, 1)]
ff_st_dict    = {'1-state':'ideal','2-state':'oring','3-state':'zhurkin','4-state':'zhurkin+oring'}
ff_st_colors  = {'1-state':'gold','2-state':'dodgerblue','3-state':'limegreen','4-state':'tomato'}
ff_st_marks   = {'1-state':'.','2-state':'.','3-state':'.','4-state':'.'}

ff_ind         = ['ideal','kabsch','olson']
ff_ind_colors  = {'ideal':'darkgreen','kabsch':'crimson','olson':'dodgerblue'}
ff_ind_marks   = {'ideal':'.','kabsch':'^','olson':'d'}

rs_bpt             = ['090','091','092','093','094','095','096','097','098','099','100',
                     '101','012','103','104','105','106','107','108','109','110']
rs_bpturn          = [9.0,9.1,9.2,9.3,9.4,9.5,9.6,9.7,9.8,9.9,10.0,
                      10.1,10.2,10.3,10.4,10.5,10.6,10.7,10.8,10.9,11.0]
rs_tw              = [round(float(360/i), 2) for i in rs_bpturn]
colors             = plt.cm.gnuplot(np.linspace(0, 1, len(rs_tw)))
rs_tw_colors       = {i:j for i in rs_tw for j in plt.cm.gnuplot(np.linspace(0, 1, len(rs_tw)))}
rs_bpturn_colors   = {i:j for i in rs_bpturn for j in plt.cm.gnuplot(np.linspace(0, 1, len(rs_tw)))}
rs_bpt_colors      = {i:j for i in rs_bpt for j in plt.cm.gnuplot(np.linspace(0, 1, len(rs_tw)))}

In [3]:
def pca_df(filename):
    df   = pd.DataFrame(columns=['x','y','z'])
    infile = open(filename, 'r')
    rfdata = infile.readlines()
    infile.close()
    rfdata = [i.rstrip('\n').split() for i in rfdata]
    N = int(rfdata[0][0])
    for j in range(0, N):
        df.at[j, ['x','y','z']]  = rfdata[ 5*j + 2]

    df = df.astype('float')
    df['vx'] = df.x - df.x.mean()
    df['vy'] = df.y - df.y.mean()
    df['vz'] = df.z - df.z.mean()

    cxx = ( (df.vx*df.vx).sum() - (df.vx.sum() * df.vx.sum()) ) / (N-1)
    cxy = ( (df.vx*df.vy).sum() - (df.vx.sum() * df.vy.sum()) ) / (N-1)
    cxz = ( (df.vx*df.vz).sum() - (df.vx.sum() * df.vz.sum()) ) / (N-1)
    cyx = ( (df.vy*df.vx).sum() - (df.vy.sum() * df.vx.sum()) ) / (N-1)
    cyy = ( (df.vy*df.vy).sum() - (df.vy.sum() * df.vy.sum()) ) / (N-1)
    cyz = ( (df.vy*df.vz).sum() - (df.vy.sum() * df.vz.sum()) ) / (N-1)
    czx = ( (df.vz*df.vx).sum() - (df.vz.sum() * df.vx.sum()) ) / (N-1)
    czy = ( (df.vz*df.vy).sum() - (df.vz.sum() * df.vy.sum()) ) / (N-1)
    czz = ( (df.vz*df.vz).sum() - (df.vz.sum() * df.vz.sum()) ) / (N-1)

    covar = np.matrix([ [cxx, cxy, cxz], [cyx, cyy, cyz], [czx, czy, czz] ])
    evals, evecs = np.linalg.eig( covar )
    idx = np.argsort(evals)[::-1][: len(evals)]
    evals = evals[idx]
    evecs = evecs[:, idx]

    adj = pd.DataFrame(np.dot( df[['vx', 'vy', 'vz']], evecs), columns=['e1','e2','e3'])
    del rfdata, df
    return adj

def fig_move(figdirectory):
    for filename in os.listdir(figdirectory):
        if not os.path.exists("figures"):
            os.mkdir("figures")
    for filename in os.listdir(figdirectory):
        if filename.endswith(".png"):
            shutil.move(figdirectory+'/'+filename, figdirectory+"/figures/"+filename)
    return

def optparameterdf(file):
    infile = open(file,'r')
    indata = infile.readlines()
    infile.close()
    indata = [i.strip('\n').split() for i in indata]
    header = indata[2:3]
    pars   = indata[3:]
    # Ensure all objects in dataframe are float64
    for k in range(0, len(pars)):
        for j, x in enumerate(pars[k]):
            try:
                pars[k][j] = float(x)
            except ValueError:
                pass
    # Generate dataframe
    df = pd.DataFrame.from_records(pars, columns=header)
    # Generate new column: 'Bend'
    for k in range(0, len(df)):
        x = float(df.loc[k, 'Tilt'])
        y = float(df.loc[k, 'Roll'])
        df.loc[k, 'Bend'] = float(np.sqrt(x**2 + y**2))
    del header, pars
    return df

## Load Global Data .csv file and analyze properties

In [6]:
# Load .csv files into dataframes
# --- Focus only on standard (stda) sequence types
df = pd.read_csv("data_col150_seq-primary_fullset", index_col=0)

col150_df = df[df.seq_type=='std']

col150_df

,incon,seq,seq_type,forcefield,tw,tot_bp,eo,eopt,eopt-tilt,eopt-roll,eopt-twist,eopt-shift,eopt-slide,eopt-rise,Wr,Tw,Lk,Rg
col01_oring_std_idt095,oring,col01,stda,1-state,9.5,150,108.981140,34.496231,9.223194,9.223180,1.602597e+01,1.194104e-02,1.194105e-02,3.280000e-08,-6.982547e-10,15.000000,15.0,81.174900
col01_oring_std_vic096,oring,col01,stda,3-state,9.6,150,162.488550,29.207090,9.230204,9.244379,1.071648e+01,8.015161e-03,7.994509e-03,2.067380e-05,1.810240e-02,14.981898,15.0,81.070734
col01_oring_std_vor107,oring,col01,stda,4-state,10.7,150,169.829282,42.028446,9.231855,9.317940,2.344349e+01,1.758247e-02,1.752167e-02,6.091080e-05,2.430201e-02,14.975698,15.0,81.035039
col01_oring_std_org100,oring,col01,stda,2-state,10.0,150,89.260973,18.446400,9.223210,9.223190,7.000000e-10,0.000000e+00,0.000000e+00,0.000000e+00,-3.395949e-12,15.000000,15.0,81.174955
col01_oring_std_vor096,oring,col01,stda,4-state,9.6,150,162.488550,29.207090,9.230204,9.244379,1.071648e+01,8.015161e-03,7.994509e-03,2.067380e-05,1.810240e-02,14.981898,15.0,81.070734
col01_oring_std_vic092,oring,col01,stda,3-state,9.2,150,199.428792,30.186290,9.229438,9.249436,1.168992e+01,8.745701e-03,8.724211e-03,2.152340e-05,1.919654e-02,16.980803,17.0,81.075448
col01_oring_std_vor100,oring,col01,stda,4-state,10.0,150,149.527352,18.467800,9.230849,9.205076,3.185439e-02,9.615500e-06,9.587200e-06,2.760000e-08,2.019771e-02,14.979802,15.0,81.058722
col01_oring_std_vic091,oring,col01,stda,3-state,9.1,150,213.138313,24.791971,9.229313,9.230408,6.322801e+00,4.724565e-03,4.713298e-03,1.128330e-05,1.863644e-02,16.981364,17.0,81.078367
col01_oring_std_kabsch,oring,col01,stda,kabsch,NaN,150,149.442574,18.928822,9.230970,9.206871,4.902739e-01,3.532463e-04,3.522162e-04,1.047400e-06,2.078563e-02,14.979214,15.0,81.055339
col01_oring_std_vic090,oring,col01,stda,3-state,9.0,150,228.865548,20.980374,9.229188,9.217581,2.529838e+00,1.883404e-03,1.879035e-03,4.365100e-06,1.808870e-02,16.981911,17.0,81.081216


In [7]:
# Plot eopt, lk, wr, and rg for the 3 Models based on GroupA and GroupB
# --- GroupA: col01, col03, col04, col02
# --- GroupB: col05 - col09

In [ ]:
for i in range(0, len(incons)):
    for j in range(0, len(inseqs1)):
        df2 = col150_df.loc[(col150_df.incon==incons[i])
                            &(col150_df.seq==inseqs1[j])
                            &(col150_df.tw != 'na')]
        
        fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(nrows=1, ncols=5, figsize=(20, 5), sharey=True)
        for key, grp in df2.groupby(['forcefield']):
            for key2, grp2 in grp.groupby(['seq_type']):
                if seqvars[0] == key2:
                    grp2.plot(kind='line', x='tw', y='eopt', c = statecolors[key], marker=statemarks[key], lw=1, legend=False, label=key, ax=ax1)
                elif seqvars[1] == key2:
                    grp2.plot(kind='line', x='tw', y='eopt', c = statecolors[key], marker=statemarks[key], lw=1, legend=False, label=key, ax=ax2)
                elif seqvars[2] == key2:
                    grp2.plot(kind='line', x='tw', y='eopt', c = statecolors[key], marker=statemarks[key], lw=1, legend=False, label=key, ax=ax3)
                elif seqvars[3] == key2:
                    grp2.plot(kind='line', x='tw', y='eopt', c = statecolors[key], marker=statemarks[key], lw=1, legend=False, label=key, ax=ax4)
                elif seqvars[4] == key2:
                    grp2.plot(kind='line', x='tw', y='eopt', c = statecolors[key], marker=statemarks[key], lw=1, legend=True, label=key, ax=ax5)
        
        ax1.set_ylim(0, 75)
        ax1.set_ylabel("Optimized Energy (kBT)")
        ax1.text(0.05, 0.95, incons[i]+': '+inseqs2[j]+' 0% C', transform=ax1.transAxes, verticalalignment='top')
        ax2.text(0.05, 0.95, incons[i]+': '+inseqs2[j]+' ~3% C', transform=ax2.transAxes, verticalalignment='top')
        ax3.text(0.05, 0.95, incons[i]+': '+inseqs2[j]+' 10% C', transform=ax3.transAxes, verticalalignment='top')
        ax4.text(0.05, 0.95, incons[i]+': '+inseqs2[j]+' 13% C', transform=ax4.transAxes, verticalalignment='top')
        ax5.text(0.05, 0.95, incons[i]+': '+inseqs2[j]+' 20% C', transform=ax5.transAxes, verticalalignment='top')
        
        ax1.set_xlabel('base-pairs/helical turn')
        ax1.set_xticks(range(0, len(grp2.tw)))
        ax1.set_xticklabels(grp2.tw, rotation=90)
        ax2.set_xlabel('base-pairs/helical turn')
        ax2.set_xticks(range(0, len(grp2.tw)))
        ax2.set_xticklabels(grp2.tw, rotation=90)
        ax3.set_xlabel('base-pairs/helical turn')
        ax3.set_xticks(range(0, len(grp2.tw)))
        ax3.set_xticklabels(grp2.tw, rotation=90)
        ax4.set_xlabel('base-pairs/helical turn')
        ax4.set_xticks(range(0, len(grp2.tw)))
        ax4.set_xticklabels(grp2.tw, rotation=90)
        ax5.set_xlabel('base-pairs/helical turn')
        ax5.set_xticks(range(0, len(grp2.tw)))
        ax5.set_xticklabels(grp2.tw, rotation=90)
        
        plt.legend(loc="upper left", bbox_to_anchor=(1.03,1), fancybox=True)
        plt.tight_layout()
        plt.savefig("col150_Eopt_"+incons[i]+"_"+inseqs2[j]+"_4TwModel", dpi=200, transparent=True, bbox_inches='tight')
        #plt.show()
        plt.clf()
        
        fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(nrows=1, ncols=5, figsize=(20, 5), sharey=True)
        for key, grp in df2.groupby(['forcefield']):
            for key2, grp2 in grp.groupby(['seq_type']):
                if seqvars[0] == key2:
                    grp2.plot(kind='line', x='tw', y='Wr', c = statecolors[key], marker=statemarks[key], lw=1, legend=False, label=key, ax=ax1)
                elif seqvars[1] == key2:
                    grp2.plot(kind='line', x='tw', y='Wr', c = statecolors[key], marker=statemarks[key], lw=1, legend=False, label=key, ax=ax2)
                elif seqvars[2] == key2:
                    grp2.plot(kind='line', x='tw', y='Wr', c = statecolors[key], marker=statemarks[key], lw=1, legend=False, label=key, ax=ax3)
                elif seqvars[3] == key2:
                    grp2.plot(kind='line', x='tw', y='Wr', c = statecolors[key], marker=statemarks[key], lw=1, legend=False, label=key, ax=ax4)
                elif seqvars[4] == key2:
                    grp2.plot(kind='line', x='tw', y='Wr', c = statecolors[key], marker=statemarks[key], lw=1, legend=True, label=key, ax=ax5)
        
        ax1.set_ylim(-0.5, 0.5)
        ax1.set_ylabel("Writhe")
        ax1.text(0.05, 0.95, incons[i]+': '+inseqs2[j]+' 0% C', transform=ax1.transAxes, verticalalignment='top')
        ax2.text(0.05, 0.95, incons[i]+': '+inseqs2[j]+' ~3% C', transform=ax2.transAxes, verticalalignment='top')
        ax3.text(0.05, 0.95, incons[i]+': '+inseqs2[j]+' 10% C', transform=ax3.transAxes, verticalalignment='top')
        ax4.text(0.05, 0.95, incons[i]+': '+inseqs2[j]+' 13% C', transform=ax4.transAxes, verticalalignment='top')
        ax5.text(0.05, 0.95, incons[i]+': '+inseqs2[j]+' 20% C', transform=ax5.transAxes, verticalalignment='top')
        
        ax1.set_xlabel('base-pairs/helical turn')
        ax1.set_xticks(range(0, len(grp2.tw)))
        ax1.set_xticklabels(grp2.tw, rotation=90)
        ax2.set_xlabel('base-pairs/helical turn')
        ax2.set_xticks(range(0, len(grp2.tw)))
        ax2.set_xticklabels(grp2.tw, rotation=90)
        ax3.set_xlabel('base-pairs/helical turn')
        ax3.set_xticks(range(0, len(grp2.tw)))
        ax3.set_xticklabels(grp2.tw, rotation=90)
        ax4.set_xlabel('base-pairs/helical turn')
        ax4.set_xticks(range(0, len(grp2.tw)))
        ax4.set_xticklabels(grp2.tw, rotation=90)
        ax5.set_xlabel('base-pairs/helical turn')
        ax5.set_xticks(range(0, len(grp2.tw)))
        ax5.set_xticklabels(grp2.tw, rotation=90)
        
        plt.legend(loc="upper left", bbox_to_anchor=(1.03,1), fancybox=True)
        plt.tight_layout()
        plt.savefig("col150_Wr_"+incons[i]+"_"+inseqs2[j]+"_4TwModel", dpi=200, transparent=True, bbox_inches='tight')
        #plt.show()
        plt.clf()
        
        fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(nrows=1, ncols=5, figsize=(20, 5), sharey=True)
        for key, grp in df2.groupby(['forcefield']):
            for key2, grp2 in grp.groupby(['seq_type']):
                if seqvars[0] == key2:
                    grp2.plot(kind='line', x='tw', y='Lk', c = statecolors[key], marker=statemarks[key], lw=1, legend=False, label=key, ax=ax1)
                elif seqvars[1] == key2:
                    grp2.plot(kind='line', x='tw', y='Lk', c = statecolors[key], marker=statemarks[key], lw=1, legend=False, label=key, ax=ax2)
                elif seqvars[2] == key2:
                    grp2.plot(kind='line', x='tw', y='Lk', c = statecolors[key], marker=statemarks[key], lw=1, legend=False, label=key, ax=ax3)
                elif seqvars[3] == key2:
                    grp2.plot(kind='line', x='tw', y='Lk', c = statecolors[key], marker=statemarks[key], lw=1, legend=False, label=key, ax=ax4)
                elif seqvars[4] == key2:
                    grp2.plot(kind='line', x='tw', y='Lk', c = statecolors[key], marker=statemarks[key], lw=1, legend=True, label=key, ax=ax5)
        
        ax1.set_ylim(12, 18)
        ax1.set_ylabel("Linking Number")
        ax1.text(0.05, 0.95, incons[i]+': '+inseqs2[j]+' 0% C', transform=ax1.transAxes, verticalalignment='top')
        ax2.text(0.05, 0.95, incons[i]+': '+inseqs2[j]+' ~3% C', transform=ax2.transAxes, verticalalignment='top')
        ax3.text(0.05, 0.95, incons[i]+': '+inseqs2[j]+' 10% C', transform=ax3.transAxes, verticalalignment='top')
        ax4.text(0.05, 0.95, incons[i]+': '+inseqs2[j]+' 13% C', transform=ax4.transAxes, verticalalignment='top')
        ax5.text(0.05, 0.95, incons[i]+': '+inseqs2[j]+' 20% C', transform=ax5.transAxes, verticalalignment='top')
        
        ax1.set_xlabel('base-pairs/helical turn')
        ax1.set_xticks(range(0, len(grp2.tw)))
        ax1.set_xticklabels(grp2.tw, rotation=90)
        ax2.set_xlabel('base-pairs/helical turn')
        ax2.set_xticks(range(0, len(grp2.tw)))
        ax2.set_xticklabels(grp2.tw, rotation=90)
        ax3.set_xlabel('base-pairs/helical turn')
        ax3.set_xticks(range(0, len(grp2.tw)))
        ax3.set_xticklabels(grp2.tw, rotation=90)
        ax4.set_xlabel('base-pairs/helical turn')
        ax4.set_xticks(range(0, len(grp2.tw)))
        ax4.set_xticklabels(grp2.tw, rotation=90)
        ax5.set_xlabel('base-pairs/helical turn')
        ax5.set_xticks(range(0, len(grp2.tw)))
        ax5.set_xticklabels(grp2.tw, rotation=90)
        
        plt.legend(loc="upper left", bbox_to_anchor=(1.03,1), fancybox=True)
        plt.tight_layout()
        plt.savefig("col150_Lk_"+incons[i]+"_"+inseqs2[j]+"_4TwModel", dpi=200, transparent=True, bbox_inches='tight')
        #plt.show()
        plt.clf()
        
        fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(nrows=1, ncols=5, figsize=(20, 5), sharey=True)
        for key, grp in df2.groupby(['forcefield']):
            for key2, grp2 in grp.groupby(['seq_type']):
                if seqvars[0] == key2:
                    grp2.plot(kind='line', x='tw', y='Rg', c = statecolors[key], marker=statemarks[key], lw=1, legend=False, label=key, ax=ax1)
                elif seqvars[1] == key2:
                    grp2.plot(kind='line', x='tw', y='Rg', c = statecolors[key], marker=statemarks[key], lw=1, legend=False, label=key, ax=ax2)
                elif seqvars[2] == key2:
                    grp2.plot(kind='line', x='tw', y='Rg', c = statecolors[key], marker=statemarks[key], lw=1, legend=False, label=key, ax=ax3)
                elif seqvars[3] == key2:
                    grp2.plot(kind='line', x='tw', y='Rg', c = statecolors[key], marker=statemarks[key], lw=1, legend=False, label=key, ax=ax4)
                elif seqvars[4] == key2:
                    grp2.plot(kind='line', x='tw', y='Rg', c = statecolors[key], marker=statemarks[key], lw=1, legend=True, label=key, ax=ax5)
        ax1.set_ylim(60, 84)
        ax1.set_ylabel("Radius of Gyration")
        ax1.text(0.05, 0.95, incons[i]+': '+inseqs2[j]+' 0% C', transform=ax1.transAxes, verticalalignment='top')
        ax2.text(0.05, 0.95, incons[i]+': '+inseqs2[j]+' ~3% C', transform=ax2.transAxes, verticalalignment='top')
        ax3.text(0.05, 0.95, incons[i]+': '+inseqs2[j]+' 10% C', transform=ax3.transAxes, verticalalignment='top')
        ax4.text(0.05, 0.95, incons[i]+': '+inseqs2[j]+' 13% C', transform=ax4.transAxes, verticalalignment='top')
        ax5.text(0.05, 0.95, incons[i]+': '+inseqs2[j]+' 20% C', transform=ax5.transAxes, verticalalignment='top')
        
        ax1.set_xlabel('base-pairs/helical turn')
        ax1.set_xticks(range(0, len(grp2.tw)))
        ax1.set_xticklabels(grp2.tw, rotation=90)
        ax1.axhline(y=round(float((3.4/(2*np.pi))*Ncirc), 2), linestyle=':', color='grey')
        ax2.set_xlabel('base-pairs/helical turn')
        ax2.set_xticks(range(0, len(grp2.tw)))
        ax2.set_xticklabels(grp2.tw, rotation=90)
        ax2.axhline(y=round(float((3.4/(2*np.pi))*Ncirc), 2), linestyle=':', color='grey')
        ax3.set_xlabel('base-pairs/helical turn')
        ax3.set_xticks(range(0, len(grp2.tw)))
        ax3.set_xticklabels(grp2.tw, rotation=90)
        ax3.axhline(y=round(float((3.4/(2*np.pi))*Ncirc), 2), linestyle=':', color='grey')
        ax4.set_xlabel('base-pairs/helical turn')
        ax4.set_xticks(range(0, len(grp2.tw)))
        ax4.set_xticklabels(grp2.tw, rotation=90)
        ax4.axhline(y=round(float((3.4/(2*np.pi))*Ncirc), 2), linestyle=':', color='grey')
        ax5.set_xlabel('base-pairs/helical turn')
        ax5.set_xticks(range(0, len(grp2.tw)))
        ax5.set_xticklabels(grp2.tw, rotation=90)
        ax5.axhline(y=round(float((3.4/(2*np.pi))*Ncirc), 2), linestyle=':', color='grey')
        
        plt.legend(loc="upper left", bbox_to_anchor=(1.03,1), fancybox=True)
        plt.tight_layout()
        plt.savefig("col150_Rg_"+incons[i]+"_"+inseqs2[j]+"_4TwModel", dpi=200, transparent=True, bbox_inches='tight')
        #plt.show()
        plt.clf()
        
        del df2, grp, grp2
        fig_move(path)

## Load .par files into dataframes and analyze